# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [8]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key
# gmaps.configure(api_key = g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [6]:
weather_df = pd.read_csv(r'../Outputs/WeatherPy_Data.csv')
weather_df

City Country  Temperature  Max Temp  Humidity %  Cloudiness %  \
0    Bila Tserkva      UA        61.29     61.29          77            99   
1           Ji’an      CN        77.23     77.23          90            30   
2     East London      ZA        51.21     51.21          66            45   
3      Port Hardy      CA        57.13     57.13          86            96   
4     Yellowknife      CA        71.29     71.29          59            84   
..            ...     ...          ...       ...         ...           ...   
584     Kisangani      CD        75.00     75.00          60            71   
585     Bariloche      AR        49.75     49.75          46             0   
586      Aripuanã      BR        82.45     82.45          41             0   
587    Codrington      AU        40.98     40.98          91           100   
588        Kalabo      ZM        56.66     56.66          34             0   

     Wind Speed (mph)  Latitude  Longitude  
0               10.85   49.8094    30.1121  
1                3.51   27.1172   114.9793  
2               21.14  -33.0153    27.9116  
3                5.75   50.6996  -127.4199  
4                8.39   62.4560  -114.3525  
..                ...       ...        ...  
584              0.96    0.5167    25.2000  
585             12.66  -41.1456   -71.3082  
586              2.48   -9.1667   -60.6333  
587              9.89  -38.2667   141.9667  
588              6.38  -14.9700    22.6814  

[589 rows x 9 columns]

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [9]:
# Store latitude and longitude in locations
locations = weather_df[["Latitude", "Longitude"]]

# Fill NaN values and convert to float
humidity = weather_df["Humidity %"].astype(float)

In [12]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=2.1)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [15]:
ideal_cities = weather_df.loc[(weather_df['Max Temp']>=65) & (weather_df['Temperature']<89) &
                (weather_df['Cloudiness %']<=10) & (weather_df['Wind Speed (mph)']<12)].reset_index()

ideal_cities

index          City Country  Temperature  Max Temp  Humidity %  \
0      26     Indiaroba      BR        70.57     70.57          89   
1      36        Lompoc      US        72.95     90.79          70   
2      68      Maragheh      IR        80.04     80.04          29   
3      69  San Patricio      US        86.54     88.52          41   
4      73        Pishin      PK        76.10     76.10          39   
..    ...           ...     ...          ...       ...         ...   
69    549        Destin      US        86.38     90.05          76   
70    552      El Campo      US        85.95     88.79          58   
71    566       Fortuna      US        66.24     82.18          65   
72    570       Colares      PT        63.48     65.28          78   
73    586      Aripuanã      BR        82.45     82.45          41   

    Cloudiness %  Wind Speed (mph)  Latitude  Longitude  
0              1              4.18  -11.5192   -37.5117  
1              1             11.50   34.6391  -120.4579  
2              0              2.21   37.3917    46.2398  
3              3              6.93   28.0170   -97.5169  
4              0              4.45   30.5803    66.9961  
..           ...               ...       ...        ...  
69             1              8.05   30.3935   -86.4958  
70             1              0.00   29.1966   -96.2697  
71             1              7.00   40.5982  -124.1573  
72             4              7.00   38.7992    -9.4469  
73             0              2.48   -9.1667   -60.6333  

[74 rows x 10 columns]

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [31]:
# Store into variable named hotel_df
# Add a "Hotel Name" column to the DataFrame.

hotel_df = ideal_cities.copy()
hotel_df["Hotel Name"] = ""
hotel_df["Hotel Rating"] = ""
hotel_df

index          City Country  Temperature  Max Temp  Humidity %  \
0      26     Indiaroba      BR        70.57     70.57          89   
1      36        Lompoc      US        72.95     90.79          70   
2      68      Maragheh      IR        80.04     80.04          29   
3      69  San Patricio      US        86.54     88.52          41   
4      73        Pishin      PK        76.10     76.10          39   
..    ...           ...     ...          ...       ...         ...   
69    549        Destin      US        86.38     90.05          76   
70    552      El Campo      US        85.95     88.79          58   
71    566       Fortuna      US        66.24     82.18          65   
72    570       Colares      PT        63.48     65.28          78   
73    586      Aripuanã      BR        82.45     82.45          41   

    Cloudiness %  Wind Speed (mph)  Latitude  Longitude Hotel Name  \
0              1              4.18  -11.5192   -37.5117              
1              1             11.50   34.6391  -120.4579              
2              0              2.21   37.3917    46.2398              
3              3              6.93   28.0170   -97.5169              
4              0              4.45   30.5803    66.9961              
..           ...               ...       ...        ...        ...   
69             1              8.05   30.3935   -86.4958              
70             1              0.00   29.1966   -96.2697              
71             1              7.00   40.5982  -124.1573              
72             4              7.00   38.7992    -9.4469              
73             0              2.48   -9.1667   -60.6333              

   Hotel Rating  
0                
1                
2                
3                
4                
..          ...  
69               
70               
71               
72               
73               

[74 rows x 12 columns]

In [33]:
# Build URL using the Google Maps API
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for index,row in hotel_df.iterrows():

    
    # rewrite params dict
    city = row["City"]
    ideal_coords = f"{row['Latitude']},{row['Longitude']}"
    params = {
        "location": ideal_coords,
        "types": "lodging",
        "radius": 5000,
        "key": g_key
    }
    
    # get response
    response_hotel = requests.get(base_url,params=params).json()
    print(f"Looking for hotels in {city}")
#     print(response_hotel)

    try:
        hotel_df.loc[index, "Hotel Name"] = response_hotel["results"][0]["name"]
        hotel_df.loc[index, "Hotel Rating"] = response_hotel["results"][0]["rating"]
        
    
    except(KeyError, IndexError):
        print(f"No hotels found in {city}")
        pass

Looking for hotels in Indiaroba
Looking for hotels in Lompoc
Looking for hotels in Maragheh
Looking for hotels in San Patricio
Looking for hotels in Pishin
Looking for hotels in Ponta do Sol
Looking for hotels in Cayenne
Looking for hotels in Tigzirt
Looking for hotels in Birjand
Looking for hotels in Fairbanks
Looking for hotels in Roebourne
Looking for hotels in Bulungu
No hotels found in Bulungu
Looking for hotels in Saint-François
Looking for hotels in Hanzhong
No hotels found in Hanzhong
Looking for hotels in Caravelas
Looking for hotels in Katsuura
Looking for hotels in Elizabeth City
Looking for hotels in Vostok
No hotels found in Vostok
Looking for hotels in Shimoda
Looking for hotels in Karratha
Looking for hotels in Morehead
Looking for hotels in Kindu
Looking for hotels in Lixouri
Looking for hotels in Creston
Looking for hotels in Arcata
Looking for hotels in Crossville
Looking for hotels in Nchelenge
Looking for hotels in Paranaíba
Looking for hotels in Misratah
Looking fo

In [34]:
# Store the first Hotel result into the DataFrame.
hotel_df

index          City Country  Temperature  Max Temp  Humidity %  \
0      26     Indiaroba      BR        70.57     70.57          89   
1      36        Lompoc      US        72.95     90.79          70   
2      68      Maragheh      IR        80.04     80.04          29   
3      69  San Patricio      US        86.54     88.52          41   
4      73        Pishin      PK        76.10     76.10          39   
..    ...           ...     ...          ...       ...         ...   
69    549        Destin      US        86.38     90.05          76   
70    552      El Campo      US        85.95     88.79          58   
71    566       Fortuna      US        66.24     82.18          65   
72    570       Colares      PT        63.48     65.28          78   
73    586      Aripuanã      BR        82.45     82.45          41   

    Cloudiness %  Wind Speed (mph)  Latitude  Longitude  \
0              1              4.18  -11.5192   -37.5117   
1              1             11.50   34.6391  -120.4579   
2              0              2.21   37.3917    46.2398   
3              3              6.93   28.0170   -97.5169   
4              0              4.45   30.5803    66.9961   
..           ...               ...       ...        ...   
69             1              8.05   30.3935   -86.4958   
70             1              0.00   29.1966   -96.2697   
71             1              7.00   40.5982  -124.1573   
72             4              7.00   38.7992    -9.4469   
73             0              2.48   -9.1667   -60.6333   

                                      Hotel Name Hotel Rating  
0                                Pura Vida Bahia          4.6  
1       Holiday Inn Express Lompoc, an IHG Hotel          4.2  
2                           Maragheh Grand Hotel          3.8  
3                             Motel 6 Sinton, TX          3.4  
4                              City Hotel Pishin          3.4  
..                                           ...          ...  
69  Inn On Destin Harbor-Ascend Hotel Collection          3.5  
70                  Lonestar Inn, El Campo Texas            4  
71                    Super 8 by Wyndham Fortuna          4.1  
72                          Arribas Sintra Hotel          4.4  
73                                                             

[74 rows x 12 columns]

In [39]:
# Plot markers on top of the heatmap.
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [38]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)

# Add layer
fig.add_layer(markers)
fig.add_layer(heat_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))